#### importing libraries

In [141]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset ,DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### building the architecture of ANNS

In [142]:
class Model(nn.Module):
    def __init__(self,in_f=4,h1_f=8,h2_f=9,out_f=3):
        super().__init__()
        self.fc1=nn.Linear(in_f,h1_f)
        self.fc2=nn.Linear(h1_f,h2_f)
        self.fc3=nn.Linear(h2_f,out_f)
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        
        return x

In [143]:
model=Model()

In [144]:
model.parameters

<bound method Module.parameters of Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (fc3): Linear(in_features=9, out_features=3, bias=True)
)>

#### calculating the parameters of architecture

In [145]:
s=0
for param in model.parameters():
    print(param.numel())
    s=s+param.numel()
print("total parameters",s)

32
8
72
9
27
3
total parameters 151


In [146]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

#### loading iris data from sklearn

In [147]:
iris=datasets.load_iris()

In [148]:
df=pd.DataFrame(iris.data)

In [149]:
df.columns=iris.feature_names

In [150]:
df['target']=iris.target

In [151]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [152]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [153]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [154]:
x=df.drop('target',axis=1).values

In [155]:
y=df['target'].values

In [156]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

In [157]:

xtrain=torch.FloatTensor(xtrain)
xtest=torch.FloatTensor(xtest)
ytrain=torch.LongTensor(ytrain)
ytest=torch.LongTensor(ytest)

In [158]:
trainloader=DataLoader(xtrain,batch_size=60,shuffle=True,pin_memory=True)
# print(trainloader)
testloader=DataLoader(xtest,batch_size=60,shuffle=True,pin_memory=True)

In [159]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)

In [160]:
import time
start=time.time()
epochs=100
losses=[]

for i in range(epochs):
    ypred=model.forward(xtrain)
    loss=criterion(ypred,ytrain)
    losses.append(loss)
    
    if i%10 ==0:
        print(f"epoch {i} loss {loss.item()}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


end=time.time()-start
print(f"total time for execution {end}")

epoch 0 loss 1.122549295425415
epoch 10 loss 0.9825970530509949
epoch 20 loss 0.7709909081459045
epoch 30 loss 0.5683454275131226
epoch 40 loss 0.3946301341056824
epoch 50 loss 0.24858637154102325
epoch 60 loss 0.13527455925941467
epoch 70 loss 0.08599849790334702
epoch 80 loss 0.06881745904684067
epoch 90 loss 0.062203481793403625
total time for execution 0.18380975723266602


#### calculating the accuracy and other metrics 

In [177]:
correct=0
with torch.no_grad():
    for i in range(len(xtest)):
        y_val=model.forward(xtest[i])
        print(f"{y_val},predicted value {y_val.argmax().item()} , actual value {ytest[i]}")
        if y_val.argmax().item()== ytest[i]:
            correct+=1
    print(f"total correct predictions {correct} out of {len(xtest)}")
    print(f"accuray ={(correct/len(xtest))*100}")
        

tensor([-4.4355,  4.8666, -0.3709]),predicted value 1 , actual value 1
tensor([  6.9815,   1.0077, -11.2205]),predicted value 0 , actual value 0
tensor([-15.9100,   3.3101,  16.5859]),predicted value 2 , actual value 2
tensor([-4.8988,  4.3594,  0.8497]),predicted value 1 , actual value 1
tensor([-5.5020,  5.0476,  0.8162]),predicted value 1 , actual value 1
tensor([  6.5073,   0.9656, -10.4687]),predicted value 0 , actual value 0
tensor([-1.2953,  3.7205, -3.1263]),predicted value 1 , actual value 1
tensor([-9.6683,  3.5463,  8.0429]),predicted value 2 , actual value 2
tensor([-7.3271,  3.9409,  4.5933]),predicted value 2 , actual value 1
tensor([-2.3763,  4.2375, -2.3042]),predicted value 1 , actual value 1
tensor([-8.0005,  3.8642,  5.4593]),predicted value 2 , actual value 2
tensor([  6.3049,   0.8523, -10.0325]),predicted value 0 , actual value 0
tensor([  6.9140,   0.8596, -10.9375]),predicted value 0 , actual value 0
tensor([  6.3738,   0.9027, -10.1952]),predicted value 0 , act

In [164]:
len(xtest)

30

In [179]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix